In [ ]:
!wget -O dataset.zip "https://partage.imt.fr/index.php/s/jJwxpEWyecC8CRW/download"

--2024-12-13 21:56:22--  https://partage.imt.fr/index.php/s/jJwxpEWyecC8CRW/download
Resolving partage.imt.fr (partage.imt.fr)... 193.48.235.12
Connecting to partage.imt.fr (partage.imt.fr)|193.48.235.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘dataset.zip’

dataset.zip             [     <=>            ]   5.04G   613KB/s    in 2h 33m  

2024-12-14 00:30:21 (572 KB/s) - ‘dataset.zip’ saved [5414284018]



In [ ]:
!unzip dataset.zip -d dataset

In [ ]:
import numpy as np
from scipy.io import wavfile
from matplotlib.mlab import specgram
import os

audio_folder = '/content/dataset'
output_folder = '/content/drive/MyDrive/Projet Commande Entreprise/Spectrograms_csv'

os.makedirs(output_folder, exist_ok=True)

nfft = 2048
noverlap = nfft // 2
step = 2099000
frequency_shift = -7e6
c = 0

# Open CSV file in append mode
output_csv = os.path.join(output_folder, "all_spectrograms.csv")
with open(output_csv, 'ab') as f:
    for filename in os.listdir(audio_folder):
        if filename.endswith('.wav'):
            file_path = os.path.join(audio_folder, filename)
            fs, data = wavfile.read(file_path)

            data_IQ = data[:, 0] + 1j * data[:, 1]
            t = np.arange(step) / fs

            segments = [data_IQ[i:i + step] for i in range(0, len(data_IQ), step)]

            for i, segment in enumerate(segments):
                Pxx, freqs, bins = specgram(segment, NFFT=nfft, Fs=fs, noverlap=noverlap)
                if Pxx.shape == (2048, 2048):
                    # Save original spectrogram
                    np.savetxt(f, Pxx.flatten()[None], delimiter=",")
                    c += 1

                    # Apply frequency shift
                    shifted_segment = segment * np.exp(1j * 2 * np.pi * frequency_shift * t[:len(segment)])
                    shifted_Pxx, shifted_freqs, _ = specgram(shifted_segment, NFFT=nfft, Fs=fs, noverlap=noverlap)

                    # Save shifted spectrogram
                    np.savetxt(f, shifted_Pxx.flatten()[None], delimiter=",")
                    c += 1
                else:
                    print(f"Skipping segment {i} of file {filename} due to unexpected Pxx shape: {Pxx.shape}")

print(f"Processed {c} spectrograms.")
print(f"All spectrogram matrices saved in {output_csv}")


In [ ]:
import shutil
shutil.copytree('/content/Spectrograms_csv', '/content/drive/My Drive/Spectrograms_csv')